# Machine Learning Language Translation from English To Persian

## Import Dependencies

In [6]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,LSTM,Dense
import numpy as np
from loguru import logger

## Initialize parameters

In [7]:
BATCH_SIZE= 64
EPOCHS=100
LATENT_DIM=256 # latent dimensionality of encoding space
NUM_SAMPLES=2600 # number of samples to train
DATA_PATH='./pes.txt'

## Preprocessing

In [8]:
# Vectorize the data
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(DATA_PATH,'r',encoding='utf-8') as file:
  lines = file.read().split('\n')
for line in lines[:min(NUM_SAMPLES,len(lines)-1)]:
  input_text, target_text , _ = line.split('\t')
  # The user use "tab" as the "start sequence" character
  # for the targets, and "\n" as "end of sequence" charecter
  target_text = '\t' + target_text + "\n"
  input_texts.append(input_text)
  target_texts.append(target_text)
  for char in input_text:
    if char not in input_characters:
      input_characters.add(char)
  for char in target_text:
    if char not in target_characters:
      target_characters.add(char)
    

In [12]:
# extract some properties
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])
logger.info(f"Number of samples: {len(input_texts)}")
logger.info(f"Number of unique input tokens: {num_encoder_tokens}", )
logger.info(f"Number of unique target tokens: {num_decoder_tokens}")
logger.info(f"Max sequence length for input: {max_encoder_seq_length}")
logger.info(f"Max sequence length for target: {max_decoder_seq_length}")

2024-04-06 02:38:33.552 | INFO     | __main__:<module>:8 - Number of samples: 2600
2024-04-06 02:38:33.554 | INFO     | __main__:<module>:9 - Number of unique input tokens: 67
2024-04-06 02:38:33.555 | INFO     | __main__:<module>:10 - Number of unique target tokens: 75
2024-04-06 02:38:33.555 | INFO     | __main__:<module>:11 - Max sequence length for input: 42
2024-04-06 02:38:33.556 | INFO     | __main__:<module>:12 - Max sequence length for target: 65


In [14]:
input_token_index = dict(
  [(char,i) for i,char in enumerate(input_characters)]
)
target_token_index = dict(
  [(char,i) for i,char in enumerate(target_characters)]
)

### one hot representation

In [15]:
encoder_input_data = np.zeros(
  (len(input_texts), max_encoder_seq_length,num_encoder_tokens),
  dtype="float32"
)
decoder_input_data = np.zeros(
  (len(input_texts), max_decoder_seq_length,num_decoder_tokens),
    dtype="float32"
)
decoder_target_data = np.zeros(
  (len(target_texts), max_decoder_seq_length,num_decoder_tokens),
  dtype="float32"
)

In [18]:
for i , (input_text, target_text) in enumerate(zip(input_texts,target_texts)):
  for t , char in enumerate(input_text):
    encoder_input_data[i,t,input_token_index[char]]=1.
  encoder_input_data[i, t+1:, input_token_index[' ']]=1.
  for t, char in enumerate(target_text):
    # decoder_target_data is ahead of decoder_input_data by one timestep
    decoder_input_data[i, t , target_token_index[char]]= 1.
    if t > 0:
      # decoder_target_data will be ahead by one timestep
      # and will not include the start character
      decoder_target_data[i, t-1, target_token_index[char]]=1.
  decoder_input_data[i, t+1:, target_token_index[' ']] = 1.
  decoder_target_data[i, t:, target_token_index[' ']] = 1.


# Model Architecture

In [32]:
class lstm_model:
  def __init__(self,
               num_encoder_tokens,num_decoder_tokens,
               latent_dim, encoder_input_data,
               decoder_input_data, decoder_target_data,
               batch_size, epochs
               ) -> None:
    self.num_encoder_tokens = num_encoder_tokens
    self.num_decoder_tokens = num_decoder_tokens
    self.latent_dim = latent_dim
    self.encoder_input_data = encoder_input_data
    self.decoder_input_data = decoder_input_data
    self.decoder_target_data = decoder_target_data
    self.batch_size = batch_size
    self.epochs = epochs
  def encoder(self):
    self.encoder_inputs = Input(shape=(None, self.num_encoder_tokens))
    encoder = LSTM(self.latent_dim, return_state=True)
    encoder_outputs, state_h, state_c = encoder(self.encoder_inputs)
    self.encoder_states = [state_h,state_c]
  def decoder(self):
    self.decoder_inputs = Input(shape=(None,self.num_decoder_tokens))
    decoder = LSTM(self.latent_dim,return_sequences=True,return_state=True)
    self.decoder_outputs,_,_ = decoder(self.decoder_inputs, initial_state=self.encoder_states)
    decoder_dense = Dense(self.num_decoder_tokens,activation="softmax")
    self.decoder_outputs = decoder_dense(self.decoder_outputs)
  def model(self):
    self.model = Model([self.encoder_inputs,self.decoder_inputs], self.decoder_outputs)
    self.model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics = ['accuracy'])
  def training(self):
    return self.model.fit([self.encoder_input_data,self.decoder_input_data],self.decoder_target_data,
                   batch_size = self.batch_size,
                   epochs= self.epochs, validation_split = 0.2)
    
    
translation_model_obj = lstm_model(
                                  num_encoder_tokens=num_encoder_tokens,
                                  num_decoder_tokens=num_decoder_tokens,
                                  latent_dim=LATENT_DIM,
                                  encoder_input_data= encoder_input_data,
                                  decoder_input_data=decoder_input_data, 
                                  decoder_target_data=decoder_target_data,
                                  batch_size=BATCH_SIZE, epochs=EPOCHS
                                   )

  

In [33]:
translation_model_obj.encoder()
translation_model_obj.decoder()
translation_model_obj.model()
history = translation_model_obj.training()

Epoch 1/100
33/33 [==============================] - 14s 372ms/step - loss: 1.8317 - accuracy: 0.6540 - val_loss: 2.0384 - val_accuracy: 0.5075
Epoch 2/100
33/33 [==============================] - 11s 327ms/step - loss: 1.3874 - accuracy: 0.6745 - val_loss: 1.9986 - val_accuracy: 0.5280
Epoch 3/100
33/33 [==============================] - 6s 177ms/step - loss: 1.2967 - accuracy: 0.6786 - val_loss: 1.9520 - val_accuracy: 0.5280
Epoch 4/100
33/33 [==============================] - 6s 182ms/step - loss: 1.2583 - accuracy: 0.6789 - val_loss: 1.8466 - val_accuracy: 0.5208
Epoch 5/100
33/33 [==============================] - 6s 177ms/step - loss: 1.2205 - accuracy: 0.6774 - val_loss: 2.0614 - val_accuracy: 0.5280
Epoch 6/100
33/33 [==============================] - 6s 177ms/step - loss: 1.1937 - accuracy: 0.6793 - val_loss: 1.8037 - val_accuracy: 0.4975
Epoch 7/100
33/33 [==============================] - 6s 175ms/step - loss: 1.1616 - accuracy: 0.6811 - val_loss: 1.7562 - val_accuracy: 0.51